In [63]:
import pandas as pd

stores_train = pd.read_csv('data/stores_train.csv')
plaace_hierarchy = pd.read_csv('data/plaace_hierarchy.csv')


stores_train_copy= stores_train.copy()

plaace_hierarchy_copy = plaace_hierarchy.copy()
plaace_hierarchy_copy.drop(columns=['sales_channel_name'], inplace=True)

plaace_hierarchy_copy['lv1'] = plaace_hierarchy_copy['lv1'].astype('category')
plaace_hierarchy_copy['lv2'] = plaace_hierarchy_copy['lv2'].astype('category')

df = stores_train_copy

df = pd.merge(df, plaace_hierarchy_copy, how='left', on='plaace_hierarchy_id')

busstops = pd.read_csv('data/busstops_norway.csv')
busstops_copy = busstops.copy()
busstops_copy['busstop_lat'] = busstops_copy['geometry'].apply(lambda x: float(x.split(' ')[1][:-1]))
busstops_copy['busstop_lon'] = busstops_copy['geometry'].apply(lambda x: float(x.split(' ')[0][6:]))

In [64]:
busstops_copy

,busstop_id,stopplace_type,importance_level,side_placement,geometry,busstop_lat,busstop_lon
0,853cb081-cc32-4880-aa3e-26e96870d874,Plattform og lomme,Mangler viktighetsnivå,LEFT_AND_RIGHT,POINT(6.54064723379766 61.0618887850553),61.061889,6.540647
1,156b052b-2771-497a-b4f4-97fed59e1aca,"Lomme og skilt, ikke plattform",Mangler viktighetsnivå,LEFT_AND_RIGHT,POINT(5.89980086113255 60.1421872817075),60.142187,5.899801
2,7312a280-e14f-4b09-a421-02e8fe1bc63e,NaN,Mangler viktighetsnivå,MIDDLE_LEFT,POINT(10.7781327278563 59.9299988828761),59.929999,10.778133
3,d9cda2c7-355a-49c1-b56c-a33180d2a82e,NaN,Mangler viktighetsnivå,MIDDLE_LEFT,POINT(10.7781496457324 59.9301044643692),59.930104,10.778150
4,f803bcd3-182d-450b-bbb4-113c6ca885c2,NaN,Mangler viktighetsnivå,MIDDLE_RIGHT,POINT(10.7781676444854 59.9302099995898),59.930210,10.778168
...,...,...,...,...,...,...,...
68390,12edd887-a122-44e2-896b-a5f663917f88,"Lomme og skilt, ikke plattform",Mangler viktighetsnivå,MIDDLE,POINT(5.85431348276154 60.2029816170228),60.202982,5.854313
68391,38dd0aee-b497-43c2-943b-51ab5c6d0eb7,"Lomme og skilt, ikke plattform",Mangler viktighetsnivå,MIDDLE,POINT(5.85387810987475 60.2029250857799),60.202925,5.853878
68392,3803f0fe-d6dc-4211-a44d-bbc3453af289,"Lomme og skilt, ikke plattform",Mangler viktighetsnivå,MIDDLE,POINT(10.6495319726142 62.9425599428163),62.942560,10.649532
68393,5b177625-d0dc-40ae-b554-e97f7d400e24,Plattform og lomme,Lokalt knutepunkt,MIDDLE,POINT(9.16421204735413 61.0634296716317),61.063430,9.164212


In [84]:
from haversine import haversine, Unit
import time
import numpy as np

from numpy import NAN

def find_closest_busstop(lat, lon):
    distances = busstops_copy.apply(lambda x: haversine((lat, lon), (x['busstop_lat'], x['busstop_lon']), unit=Unit.METERS), axis=1)
    return distances.min()

for i, row in stores_train_copy.iterrows():
    
    if(stores_train_copy.loc[i, 'distance_to_closest_busstop'] == np.NaN):
        
        print(i)
        distance = find_closest_busstop(row['lat'], row['lon'])
        
        stores_train_copy.loc[i, 'distance_to_closest_busstop'] = find_closest_busstop(row['lat'], row['lon'])
        
#stores_train_copy = pd.to_csv('data/stores_train_with_busstops.csv')

In [85]:
stores_train_copy

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue,distance_to_closest_busstop
0,983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998,495.653643
1,987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,MCDONALDS,NaN,23.828,102.355657
2,984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,16.099,25.531803
3,914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,9.296,52.093129
4,913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,BURGER KING,Tillertorget,4.528,166.165890
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12854,915789943-915806929-781991,2016,MEIERIGÅRDEN BRYGGERIUTSALG,2.8.11.2,Beer and soda shop,7010705,THUEGATA 2,59.416276,10.480970,NaN,NaN,0.088,NaN
12855,917921733-917982368-868081,2016,GULATING ØLUTSALG CC GJØVIK,2.8.11.2,Beer and soda shop,5020406,NaN,60.799991,10.693635,GULATING GRUPPEN,CC Gjøvik,1.816,NaN
12856,911721961-911764474-496764,2016,GULATING ØLUTSALG STRØMMEN,2.8.11.2,Beer and soda shop,2310803,STØPERIVEIEN 6,59.946562,11.007659,GULATING GRUPPEN,Strømmen Storsenter,38.225,NaN
12857,914337046-914343372-721294,2016,DET GODE BRYGG,2.8.11.2,Beer and soda shop,11020113,VÅGSGATA 16,58.850261,5.735674,NaN,Bystasjonen,3.642,NaN
